# Filter and Group Trial Results: Time Series + Scalars

This cookbook shows how to:
- filter patients with `get_filtered_patients`
- download time series with `get_timeseries_as_dataframe`
- download grouped scalars with `get_trial_scalars_with_filter_and_groups_as_dataframe`

Linked resource: [Jinko](https://jinko.ai/tr-pPqG-BqG2).

In [ ]:
# Jinko specifics imports & initialization
# Please fold this section and do not change
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)
jinko.initialize()

In [ ]:
# Cookbook specifics imports
import json
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

# Cookbook parameters
# @param {"name":"trialId", "type": "string"}
# Fill the short Id of your Trial (ex: tr-EKRx-3HRt)
trialId = "tr-pPqG-BqG2"

# Optional: limit how many time series to download in this demo
max_time_series = 5

# Filter parameters (for get_filtered_patients)
arm_filter = "iv-1-30"
filter_descriptor_id = "Lymph.Drug.max"
filter_operator = "Lte"
filter_value = 5e-3

# Group parameters (for get_trial_scalars_with_filter_and_groups_as_dataframe)
group_descriptor_id = "Lymph.tmin"
group_buckets = 2

# Scalar descriptors to download
scalar_ids = ["Blood.Drug-at-P1W3D"]

# Scalar descriptors to download along with their arms
scalar_ids_on_specific_arms = {"Blood.Drug-at-P1W3D": ["iv-1-10"]}

## 1) Load the trial snapshot

In [ ]:
if not trialId:
    raise Exception("Please specify a Trial Id")

# Convert short Id to coreItemId
try:
    core_item = jinko.get_core_item_id(trialId, 1)
except Exception:
    print("Failed to find corresponding trial, check the trialId")
    raise

# List all Trial versions (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-status/post)
try:
    trial_versions = jinko.make_request(
        f"/core/v2/trial_manager/trial/{core_item['id']}/status"
    ).json()
    print(f"Fetched {len(trial_versions)} versions for the trial.")
except Exception as e:
    print(f"Error fetching trial versions: {e}")
    raise

# Get the latest completed version
latest_completed_version = next(
    (item for item in trial_versions if item["status"] == "completed"), None
)
if latest_completed_version is None:
    raise Exception("No completed Trial version found")

simulation_id = latest_completed_version["simulationId"]
trial_core_id = simulation_id["coreItemId"]
trial_snapshot_id = simulation_id["snapshotId"]

print(
    "Using completed simulation:",
    json.dumps(latest_completed_version, indent=1),
)

## 2) Display a results summary

In [ ]:
response_summary = jinko.get_trial_scalars_summary(
    trial_core_id, trial_snapshot_id, print_summary=True
)

arm_names = response_summary["arms"]

# Store the list of scenario descriptors
scenario_descriptors = [
    scalar["id"]
    for scalar in (response_summary["scalars"] + response_summary["categoricals"])
    if "ScenarioOverride" in scalar["type"]["labels"]
]
print("List of scenario overrides:\n", scenario_descriptors)

## 3) Define filters and groups

In [ ]:
# filter type definition: https://doc.jinko.ai/api/#/schemas/Filter
filter_on_scalar = [
    {
        "contents": [
            {
                "arm": arm_filter,
                "descriptorId": filter_descriptor_id,
                "operator": filter_operator,
                "value": filter_value,
            }
        ],
        "tag": "DescriptorFilter",
    }
]

# group type definition: https://doc.jinko.ai/api/#/schemas/GroupBy
group = [
    {
        "contents": {
            "arm": "crossArms",
            "descriptorId": group_descriptor_id,
            "mode": "Buckets",
            "parameters": group_buckets,
        },
        "tag": "Scalar",
    }
]

## 4) Get filtered patients list

In [ ]:
patient_ids_to_keep = jinko.get_filtered_patients(
    trial_core_id, trial_snapshot_id, filter_on_scalar  # type: ignore
)
print(f"Filtered patients: {len(patient_ids_to_keep)}")

In [ ]:
patient_ids_to_keep

## 5) Download time series

In [ ]:
# Retrieve time series ids (https://doc.jinko.ai/api/#/paths/core-v2-trial_manager-trial-trialId--snapshots--trialIdSnapshot--output_ids/get)
response = jinko.make_request(
    f"/core/v2/trial_manager/trial/{trial_core_id}/snapshots/{trial_snapshot_id}/output_ids",
    method="GET",
)
available_time_series = json.loads(response.content.decode("utf-8"))
print("Available time series:")
print(available_time_series, "")

ids_for_time_series = [x["id"]
                       for x in available_time_series][:max_time_series]
print("Using time series ids:", ids_for_time_series)

In [ ]:
df_time_series = jinko.get_timeseries_as_dataframe(
    trial_core_id,
    trial_snapshot_id,
    {ts_id: arm_names for ts_id in ids_for_time_series},
    patient_ids_to_keep,
)

display(df_time_series)

## 6) Download grouped scalar results

In [ ]:
df_scalars = jinko.get_trial_scalars_with_filter_and_groups_as_dataframe(
    trial_core_id,
    trial_snapshot_id,
    scalar_ids,
    filter_on_scalar,  # type: ignore
    group,  # type: ignore
)

display(df_scalars)

## 7) Download grouped scalar results but only on one arm

In [ ]:
df_scalars = jinko.get_trial_scalars_with_filter_and_groups_as_dataframe(
    trial_core_id,
    trial_snapshot_id,
    scalar_ids_on_specific_arms,
    filter_on_scalar,  # type: ignore
    group,  # type: ignore
)

display(df_scalars)